In [8]:
# import selenium
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from collections import deque
import logging
from joblib import Memory

# # Create a memory-based cache with a size limit of 100 MB
cache = Memory(location='~/.cache')

logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.DEBUG)
logger = logging.getLogger(__name__)

In [2]:
root_url = "https://pleasantontx.civicweb.net"
document_tree_suffix = "/filepro/documents/"
HEADERS = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.125 Safari/537.36'}

In [3]:

    
def get_soup(url_append, root_url=root_url, headers=HEADERS):
    # TODO: implement caching using joblib
    # cached_result = cache.get('html:%s' % url)
    # if cached_result:
    #     return cached_result
    url = root_url + url_append
    print(f"Getting BeautifulSoup object from {url}")
    try:
        page = requests.get(url, headers=headers)
        # cache.set('html:%s' % url, page)
        bs = BeautifulSoup(page.content,'html.parser')
        return bs
    except Exception as e:
        print("Error: ", e)
        return None
    
def get_folders(bs:BeautifulSoup, folders:deque):
    for folder in bs.find_all("a", class_='folder-link'):
        folder_name = folder.get('href')
        if folder_name not in folders:
            folders.append(folder_name)
    return folders

def get_documents(bs:BeautifulSoup, documents:list):
    # TODO: get all documents
    pass
    # for document in bs.find_all("a", class_='document-link'):
    #     if document.get('href') not in documents:
    #         documents.append(document)
    # return documents

def download_pdf(url:str, headers=HEADERS):
    page = requests.get(url, headers=headers)
    pdf = open("pdf"+str(i)+".pdf", 'wb')
    pdf.write(page.content)
    pdf.close()
    return None

In [4]:
folders = deque([])
done_folders = deque([])

In [17]:
documents = []

In [5]:
bs = get_soup(url_append=document_tree_suffix, root_url=root_url)
folders = get_folders(bs, folders)

Getting BeautifulSoup object from https://pleasantontx.civicweb.net/filepro/documents/


DEBUG:Starting new HTTPS connection (1): pleasantontx.civicweb.net:443
DEBUG:https://pleasantontx.civicweb.net:443 "GET /filepro/documents/ HTTP/1.1" 200 None


<Response [200]>
Adding new folder /filepro/documents/1009
Adding new folder /filepro/documents/108
Adding new folder /filepro/documents/109


In [10]:
while len(folders)>0:
    
    print(folders)
    print(done_folders)
    folder_name = folders.pop()
    print(f"Searching folder {folder_name}...")
    try:
    
        bs = get_soup(url_append=folder_name, root_url=root_url)

        # find additional subfolders
        initial_folders = len(folders)
        folders = get_folders(bs, folders)

        # find documents in this folder
        initial_documents = len(documents)
        documents = get_documents(bs, documents)
        
        print(f"Found {len(folders)-initial_folders} new folders and {len(documents)-initial_documents} new documents.")
    except Exception as e:
        logger.error(f"Scraping Incomplete for folder {folder_name} with Error: {e}")
        break

    done_folders.append(folder_name)
    print("done")

deque(['/filepro/documents/1009', '/filepro/documents/108'])
deque([])
Searching folder /filepro/documents/108...
Getting BeautifulSoup object from https://pleasantontx.civicweb.net/filepro/documents/108


DEBUG:Starting new HTTPS connection (1): pleasantontx.civicweb.net:443
DEBUG:https://pleasantontx.civicweb.net:443 "GET /filepro/documents/108 HTTP/1.1" 301 None
DEBUG:https://pleasantontx.civicweb.net:443 "GET /filepro/documents/108/ HTTP/1.1" 200 None
DEBUG:Starting new HTTPS connection (1): pleasantontx.civicweb.net:443


<Response [200]>
Adding new folder /filepro/documents/1688
Found 1 new folders and 0 new documents.
done
deque(['/filepro/documents/1009', '/filepro/documents/1688'])
deque(['/filepro/documents/108'])
Searching folder /filepro/documents/1688...
Getting BeautifulSoup object from https://pleasantontx.civicweb.net/filepro/documents/1688


DEBUG:https://pleasantontx.civicweb.net:443 "GET /filepro/documents/1688 HTTP/1.1" 301 None
DEBUG:https://pleasantontx.civicweb.net:443 "GET /filepro/documents/1688/ HTTP/1.1" 200 None
DEBUG:Starting new HTTPS connection (1): pleasantontx.civicweb.net:443


<Response [200]>
Found 0 new folders and 0 new documents.
done
deque(['/filepro/documents/1009'])
deque(['/filepro/documents/108', '/filepro/documents/1688'])
Searching folder /filepro/documents/1009...
Getting BeautifulSoup object from https://pleasantontx.civicweb.net/filepro/documents/1009


...

<Response [200]>
Found 0 new folders and 0 new documents.
done
deque(['/filepro/documents/5869', '/filepro/documents/5690'])
deque(['/filepro/documents/108', '/filepro/documents/1688', '/filepro/documents/1009', '/filepro/documents/1021', '/filepro/documents/1022', '/filepro/documents/3606', '/filepro/documents/3997', '/filepro/documents/5664', '/filepro/documents/5684', '/filepro/documents/5687', '/filepro/documents/3879', '/filepro/documents/3880', '/filepro/documents/5665', '/filepro/documents/5685', '/filepro/documents/5689', '/filepro/documents/5691', '/filepro/documents/5682', '/filepro/documents/5683', '/filepro/documents/5686', '/filepro/documents/5688'])
Searching folder /filepro/documents/5690...
Getting BeautifulSoup object from https://pleasantontx.civicweb.net/filepro/documents/5690


DEBUG:https://pleasantontx.civicweb.net:443 "GET /filepro/documents/5690 HTTP/1.1" 301 None
DEBUG:https://pleasantontx.civicweb.net:443 "GET /filepro/documents/5690/ HTTP/1.1" 200 None
DEBUG:Starting new HTTPS connection (1): pleasantontx.civicweb.net:443


<Response [200]>
Found 0 new folders and 0 new documents.
done
deque(['/filepro/documents/5869'])
deque(['/filepro/documents/108', '/filepro/documents/1688', '/filepro/documents/1009', '/filepro/documents/1021', '/filepro/documents/1022', '/filepro/documents/3606', '/filepro/documents/3997', '/filepro/documents/5664', '/filepro/documents/5684', '/filepro/documents/5687', '/filepro/documents/3879', '/filepro/documents/3880', '/filepro/documents/5665', '/filepro/documents/5685', '/filepro/documents/5689', '/filepro/documents/5691', '/filepro/documents/5682', '/filepro/documents/5683', '/filepro/documents/5686', '/filepro/documents/5688', '/filepro/documents/5690'])
Searching folder /filepro/documents/5869...
Getting BeautifulSoup object from https://pleasantontx.civicweb.net/filepro/documents/5869


DEBUG:https://pleasantontx.civicweb.net:443 "GET /filepro/documents/5869 HTTP/1.1" 301 None
DEBUG:https://pleasantontx.civicweb.net:443 "GET /filepro/documents/5869/ HTTP/1.1" 200 None


<Response [200]>
Found 0 new folders and 0 new documents.
done


In [12]:
print(folders)
print(done_folders)

deque([])
deque(['/filepro/documents/108', '/filepro/documents/1688', '/filepro/documents/1009', '/filepro/documents/1021', '/filepro/documents/1022', '/filepro/documents/3606', '/filepro/documents/3997', '/filepro/documents/5664', '/filepro/documents/5684', '/filepro/documents/5687', '/filepro/documents/3879', '/filepro/documents/3880', '/filepro/documents/5665', '/filepro/documents/5685', '/filepro/documents/5689', '/filepro/documents/5691', '/filepro/documents/5682', '/filepro/documents/5683', '/filepro/documents/5686', '/filepro/documents/5688', '/filepro/documents/5690', '/filepro/documents/5869'])


In [21]:
# https://pleasantontx.civicweb.net/filepro/documents/5688/ #has 4 documents
bs = get_soup(url_append='/filepro/documents/5688/', root_url=root_url)


Getting BeautifulSoup object from https://pleasantontx.civicweb.net/filepro/documents/5688/


DEBUG:Starting new HTTPS connection (1): pleasantontx.civicweb.net:443
DEBUG:https://pleasantontx.civicweb.net:443 "GET /filepro/documents/5688/ HTTP/1.1" 200 None


<Response [200]>


In [22]:
documents = get_documents(bs, documents)

In [37]:
rows = []
for doc in bs.find_all("a", class_='document-link'):
    document_info = {
        "name": doc.text.strip(),
        "subdirectory_url": doc["href"],
        "subdomain": root_url.split("https://")[-1].split(".civicweb.net")[0],
        }
    rows.append(document_info)
print(rows)

[{'name': '07-16-20 Workshop', 'subdirectory_url': '/document/5626', 'subdomain': 'pleasantontx'}, {'name': '08-06-20 Workshop', 'subdirectory_url': '/document/5628', 'subdomain': 'pleasantontx'}, {'name': '06-18-20 Workshop', 'subdirectory_url': '/document/5624', 'subdomain': 'pleasantontx'}, {'name': '02-20-20 Workshop', 'subdirectory_url': '/document/5613', 'subdomain': 'pleasantontx'}]


In [23]:
documents

[]

In [ ]:
# for doc in [document["name"] for document in rows]:
#     page = requests.get(url, headers=HEADERS)
#     pdf = open("pdf"+str(i)+".pdf", 'wb')
#     pdf.write(page.content)
#     pdf.close()